In [1]:
import pkuseg
from tqdm import tqdm

seg = pkuseg.pkuseg()           # 以默认配置加载模型
text = seg.cut('我爱北京天安门')  # 进行分词
print(text)

['我', '爱', '北京', '天安门']


In [2]:
import os

In [4]:
article_names = os.listdir('data')

In [7]:
article_contents = list()
for article_name in article_names:
    file_name = 'data/' + article_name
    file = open(file_name, 'r')
    article_content = file.read()
    file.close()
    article_contents.append(article_content)

In [13]:
article_words = list()
for i in tqdm(range(len(article_contents))):
    article_content = article_contents[i]
    words = seg.cut(article_content)
    article_words.append(words)

100%|██████████| 73/73 [00:18<00:00,  4.01it/s]


In [20]:
def counter(l: list) -> dict:
    stats = dict()
    for i in range(len(l)):
        item = l[i]
        if item not in stats:
            stats[item] = 1
        else:
            stats[item] = stats[item] + 1
    
    return stats

In [21]:
article_stats = list()
for i in tqdm(range(len(article_words))):
    article_stats.append(counter(article_words[i]))

100%|██████████| 73/73 [00:00<00:00, 609.49it/s]


In [26]:
all_term_stats = dict()
for i in tqdm(range(len(article_stats))):
    article_stat = article_stats[i]
    for word, count in article_stat.items():
        if word not in all_term_stats:
            all_term_stats[word] = count
        else:
            all_term_stats[word] = all_term_stats[word] + count

100%|██████████| 73/73 [00:00<00:00, 2058.82it/s]


In [28]:
all_term_vector = list(all_term_stats.keys())

In [31]:
doc_matrix = list()
for i in tqdm(range(len(article_stats))):
    stat_of_this_article = article_stats[i]
    doc_matrix_row = list()
    for term in all_term_vector:
        if term in stat_of_this_article:
            count = stat_of_this_article[term]
        else:
            count = 0
        doc_matrix_row.append(count)
    
    doc_matrix.append(doc_matrix_row)

100%|██████████| 73/73 [00:00<00:00, 190.16it/s]


In [33]:
len(doc_matrix)

73

In [34]:
len(doc_matrix[0])

14992

In [35]:
import pandas as pd

In [38]:
article_indexes = pd.DataFrame({
    'article_name': article_names
})

In [39]:
term_indexes = pd.DataFrame({
    'term': all_term_vector
})

In [40]:
article_indexes.to_csv('termdocmatrix/article_indexes.csv', index=False)

In [41]:
term_indexes.to_csv('termdocmatrix/term_indexes.csv', index=False)

In [42]:
import numpy as np

In [53]:
np.savetxt(
    fname = 'termdocmatrix/doc_matrix.txt',
    X = np.array(doc_matrix).astype(int),
    delimiter = ',',
    newline = '\n',
    fmt = '%u'
)